<a href="https://colab.research.google.com/github/Dinesha1999/Pdf_Query_LLM/blob/main/PDF_Query_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.6/987.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.7/371.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.6 MB/s eta 0:00:00


In [ ]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone ,Weaviate ,FAISS

In [ ]:
#getting my API key from openai
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
#connect my google drive
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
root_dir ="content/gdrive/My Drive/"

Mounted at /content/gdrive


In [ ]:
#location of the pdf file(read the file from putting the pathway of the folder )
reader = PdfReader("/content/gdrive/My Drive/PDF/Kanneliya_forest.pdf")

In [ ]:
reader

In [ ]:
#read data from the file and put them into a variable called raw_text(go to content of each page and read the text from each pages and simply return into the raw text )
raw_text = ""
for i,page in enumerate(reader.pages):
  text = page.extract_text()
  if text:
    raw_text += text


In [ ]:
raw_text

'See discussions, st ats, and author pr ofiles f or this public ation at : https://www .researchgate.ne t/public ation/377415031\nCommu nity Perception of Ecotou rism in Kanneliya Rainforest Area\nConf erence Paper  · Januar y 2024\nCITATION\n1READS\n149\n1 author:\nPathir age Kasun Udar a Perera\nUva Wellassa Univ ersity\n15 PUBLICA TIONS \xa0\xa0\xa04 CITATIONS \xa0\xa0\xa0\nSEE PROFILE\nAll c ontent f ollo wing this p age was uplo aded b y Pathir age Kasun Udar a Perera on 16 Januar y 2024.\nThe user has r equest ed enhanc ement of the do wnlo aded file.291 \n  \n* Corresponding author:  udarakasun864@gmail.com  \n  \nCommunity Perception of  \nEcotourism in Kanneliya Rainforest Area  \n \nPerera  P. K. U .* \n \n*Department of Tourism and Hospitality Management, Rajarata University of Sri \nLanka  \n \n \n \nA B S T R A C T  \n \nEcotourism is “Responsible travel to natural areas that conserve the environment, \nsustain the wellbeing of the local people and involves the interpretat

In [ ]:
raw_text[:100]

'See discussions, st ats, and author pr ofiles f or this public ation at : https://www .researchgate.'

In [ ]:
#Then need to split the text that we read into smaller chunks and we cannot hit the token size limits and use as chunk size has 1000 characters. We use the overlapping
#because first sentence we wii have the thousand characters and the second sentence start from the 800 chracters
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
len(texts)

60

In [ ]:
texts[0]

'See discussions, st ats, and author pr ofiles f or this public ation at : https://www .researchgate.ne t/public ation/377415031\nCommu nity Perception of Ecotou rism in Kanneliya Rainforest Area\nConf erence Paper  · Januar y 2024\nCITATION\n1READS\n149\n1 author:\nPathir age Kasun Udar a Perera\nUva Wellassa Univ ersity\n15 PUBLICA TIONS \xa0\xa0\xa04 CITATIONS \xa0\xa0\xa0\nSEE PROFILE\nAll c ontent f ollo wing this p age was uplo aded b y Pathir age Kasun Udar a Perera on 16 Januar y 2024.\nThe user has r equest ed enhanc ement of the do wnlo aded file.291 \n  \n* Corresponding author:  udarakasun864@gmail.com  \n  \nCommunity Perception of  \nEcotourism in Kanneliya Rainforest Area  \n \nPerera  P. K. U .* \n \n*Department of Tourism and Hospitality Management, Rajarata University of Sri \nLanka  \n \n \n \nA B S T R A C T  \n \nEcotourism is “Responsible travel to natural areas that conserve the environment, \nsustain the wellbeing of the local people and involves the interpretat

In [ ]:
texts[1]

'Lanka  \n \n \n \nA B S T R A C T  \n \nEcotourism is “Responsible travel to natural areas that conserve the environment, \nsustain the wellbeing of the local people and involves the interpretation and \neducation”. It is a well constituted and remunerative concept for numerous regional \ndevelopment strategies. In Sri Lankan context it is very famous among tourists and \ntourism business as a result of rich ecosystems throughout the country. Although eco-\ntourism popular within the context, that have been argued a lot from the perspectives \nby communities , residents and tourist.   The relationship among perception of eco -\ntourism, interpretation needs and satisfaction from the perspective of visitors and \nhosts are remained unclear. Accordingly, the current study aimed at eco -tourism \nvisitors and host commun ity of the Kanneliya rainforest area to investigative their \neco-tourism perception, expectation and barriers to the developing ecotourism in the'

In [ ]:
#download embeddings from OPENAI(embeddings is simplya set of numbers of a floating numbers,it can be use to measure the distance between two diffent text strings or sentences )
embeddings = OpenAIEmbeddings()

In [ ]:
#To find the embeddings of our text chunks we are using this vector database .It will take the text chunks and finding the corresponding embeddings and that will be stored here in the docsearch
docsearch = FAISS.from_texts(texts,embeddings)


In [ ]:
docsearch

In [ ]:
#import question answer chain from lang chain and the corresponding openai object
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [ ]:
#Here pass on different models that I need.ex:GPT-3 models and this will create a chain
chain = load_qa_chain(OpenAI(),chain_type="stuff")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [ ]:
#Now start asking questions.The given query is finding out which is the closest text in the document semantically and run the chain on it give us a response
query = "what is the forest?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs,question=query)


' The forest is the Kanneliya forest, which is a rainforest in Sri Lanka that is facing degradation due to overutilization of resources and population growth. It is also home to endemic and endangered species of animals and plants. '

In [ ]:
query ="what is the Reserve located distric of kanneliya forest?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs,question=query)

' \n\nThe Kanneliya Forest Reserve is located in the Galle district of Sri Lanka.'

In [ ]:
query ="tell about method of data collection?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs,question=query)

" The primary method of data collection used in this research study is interviews, which are conducted in an in-depth and semi-structured manner. Additionally, secondary data sources such as publications by local and international researchers are also used. The target population for this study is the community living in peripheral villages near the Kanneliya rainforest, and the sample is selected using purposive sampling methods. The use of interviews as a data collection method allows for a better understanding of the topic and the respondent's experiences, while questionnaires are used to further investigate their responses. The data is then analyzed using descriptive analysis methods, which are appropriate for qualitative research studies."